# LangChain Integration With Hugging Face

**Hugging Face** has provided the means to use language models with zero cost for API use. **Lang Chain** is a library that makes it possible to create LLM applications by integrating it with you software. 

This notebook makes it possible to learn how to integrate Hugging Face models with LangChain to create your own specialized LLMs with minimal dependency on the commercial paywall.

In [ ]:
from langchain_huggingface import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="microsoft/Phi-3-mini-4k-instruct",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 200,
        "top_k": 50,
        "temperature": 0.1,
    },
)
llm.invoke("What is DOST?")


## Using Prompt Templates

Prompt templates allow the system to format the output based on the instruction inside the prompt template

In [ ]:

#* Few shot prompt template

from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate

examples = [
  {
    "question": "What is the ASTHRD Scholarship?",
    "answer": """The Accelerated Science and Technology Human Resource Development Program (ASTHRDP) scholarship
    of the Department of Science and Technology (DOST) for individuals intending to pursue Master’s and Doctoral programs in priority S&T areas
    in the country. Scholars are encouraged to choose topics for their thesis/dissertation along the following areas:
    
    1. Climate Change and Disaster Preparedness
    2. Materials Science and Nanotechnology
    3. Natural Products and Drug Development
    """
  },
  {
    "question": "What is the scope of the examination for the DOST scholarship",
    "answer": """Assuming you are referring to the undergraduate scholraship and the Junior Level Science Scholarship (JLSS), the subjects involved are the following:
    1. Logical Reasoning - Situational questions, logical thinking, critical analysis, problem-solving
    2. English - Grammar, reading comprehension, parts of a sentence, figures of speech
    3. Science - Biology, Physics, Chemistry, Earth Scince, Advanced Sciences such as Environmental Science, Electronics
    4. Mathematics - Algebra, Basic Calculus, Precalculus, Statistics and Probability, Numerical Methods
    5. Mechanical-Technical - More on situations involving engineering principles and the emerging technologies today
    6. Self-Inventory - Soft skills such as collaboration, communication, adaptiveness, among other skills
    """
  }
]

# Example prompt of Question and Answer
example_prompt = PromptTemplate.from_template("Question: {question}\n\n{answer}")

# Create the few-shot prompt
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

llm_chain = prompt_template | llm #* pipe the prompt_template into the llm
#print(llm_chain.invoke({"input":"What is the ERDT Scholarship of DOST?"})) #Currently in debug

## Building Chains

Chains is chaining activities from a prompt to a model to a parser, and repeat.

In [ ]:
# Create a prompt template that takes an input activity
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

# Create a prompt template that places a time constraint on the output
time_prompt = PromptTemplate(
    input_variables = ['learning_plan'],
    template="I only have one week. Can you create a plan to help me hit this goal: {learning_plan}."
)

# Invoke the learning_prompt with an activity
print(learning_prompt.invoke({"activity": "play golf"}))

In [ ]:

#* Another chain using LCEL (LangChain Expression Language)
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

time_prompt = PromptTemplate(
    input_variables=["learning_plan"],
    template="I only have one week. Can you create a concise plan to help me hit this goal: {learning_plan}."
)

# Complete the sequential chain with LCEL
seq_chain = ({"learning_plan": learning_prompt | llm | StrOutputParser()}
    | time_prompt
    | llm
    | StrOutputParser())

# Call the chain
print(seq_chain.invoke({"activity": "Destroying Russia"}))

## Agents

Agents is the ise of LLMS to take action. It uses **tools** whic hare functions to extract a data the agent needs in order to respond.

The most fundamental agent is a **ReAct** agent which combines a _Reason_ with an _Action_.

In [ ]:
# Define the tools
tools = load_tools(["wikipedia"])

# Define the agent
agent = create_react_agent(llm, tools)

# Invoke the agent
response = agent.invoke({"messages": [("human", "How many people live in New York City?")]})
print(response['messages'][-1].content)


## Using Tools

LangChain agents uses tools to request data from, and in return use the data to make a response to the user. Below is a sample invokation for an agent using the tool "Wikipedia"

In [ ]:
# Define a function to retrieve customer info by-name
def retrieve_customer_info(name: str) -> str:
    """Retrieve customer information based on their name."""
    # Filter customers for the customer's name
    customer_info = customers[customers['name'] == name]
    return customer_info.to_string()
  
# Call the function on Peak Performance Co. (this is now known as a tool)
print(retrieve_customer_info("Peak Performance Co."))

In [ ]:
@tool # Converts the function into a tool
def retrieve_customer_info(name: str) -> str:
    """Retrieve customer information based on their name."""
    customer_info = customers[customers['name'] == name]
    return customer_info.to_string()

# Create a ReAct agent
agent = create_react_agent(llm,[retrieve_customer_info])

# Invoke the agent on the input
messages = agent.invoke({"messages": [("human", "Create a summary of our customer: Peak Performance Co.")]})
print(messages['messages'][-1].content)

# Retreival Augmented Generation

The steps are as follows:
1. Document Loaders - Docling most versatile as it can handle any document.
2. Data splitting - **Chunk overlap** is necessary to maintain context
    - `\n\n` - Paragraph split
    - `\n` - Sentence split
3. Storage + Retrieval (Vector databases)

### Document Loading

In [ ]:
from langchain_community.document_loaders import UnstructuredHTMLLoader
#* Document Loaders

# Create a document loader for unstructured HTML
loader = UnstructuredHTMLLoader('white_house_executive_order_nov_2023.html')

# Load the document
data = loader.load()

# Print the first document
print(data[0])

# Print the first document's metadata
print(data[0].metadata)

### Splitter

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

#* Text splitter
quote = 'Words are flowing out like endless rain into a paper cup,\nthey slither while they pass,\nthey slip away across the universe.'
chunk_size = 24
chunk_overlap = 10

# Create an instance of the splitter class
splitter = CharacterTextSplitter('\n',chunk_size=chunk_size,chunk_overlap=chunk_overlap)

# Split the string and print the chunks
docs = splitter.split_text(quote)
print(docs)
print([len(doc) for doc in docs])

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

#* Recursive splitter allows you to split the string based on a sequence of escape characters
# Here, it first splits by \n, then by whitespace, then by a null string.

quote = 'Words are flowing out like endless rain into a paper cup,\nthey slither while they pass,\nthey slip away across the universe.'
chunk_size = 24
chunk_overlap = 10

# Create an instance of the splitter class
splitter = RecursiveCharacterTextSplitter(['\n'," ",""],chunk_size=chunk_size,chunk_overlap=chunk_overlap)

# Split the document and print the chunks
docs = splitter.split_text(quote)
print(docs)
print([len(doc) for doc in docs])

### Storing and Retrieval in Vector Databases

See [this link](https://python.langchain.com/docs/integrations/vectorstores/chroma/) for an example with Chroma. Other vector stores are also available.

In [ ]:
loader = PyPDFLoader('rag_vs_fine_tuning.pdf')
data = loader.load()

# Split the document using RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=50)
docs = splitter.split_documents(data)

# Embed the documents in a persistent Chroma vector database
embedding_function = OpenAIEmbeddings(api_key='<OPENAI_API_TOKEN>', model='text-embedding-3-small')
vectorstore = Chroma.from_documents(
    docs,
    embedding=embedding_function,
    persist_directory=os.getcwd()
)

# Configure the vector store as a retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

In [ ]:
#Then, create Prompt Template
# Add placeholders to the message string
message = """
Answer the following question using the context provided:

Context:
{context}

Question:
{question}

Answer:
"""

# Create a chat prompt template from the message string
prompt_template = ChatPromptTemplate.from_messages([("human", message)])

In [ ]:
vectorstore = Chroma.from_documents(
    docs,
    embedding=OpenAIEmbeddings(api_key='<OPENAI_API_TOKEN>', model='text-embedding-3-small'),
    persist_directory=os.getcwd()
)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# Create a chain to link retriever, prompt_template, and llm
rag_chain = ({"context": retriever, "question": RunnablePassthrough()}
            | prompt_template
            | llm)

# Invoke the chain
response = rag_chain.invoke("Which popular LLMs were considered in the paper?")
print(response.content)